In [ ]:
# # Comment it out and add the files to the current directory
# from google.colab import drive
# drive.mount('/content/drive/')
# import sys
# sys.path.insert(0,'./drive/MyDrive/Colab_Notebooks/GMDL/GMDL_HW5')

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wandb

wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key f

True

In [31]:
from networks.openSetClassifier import openSetClassifier
from train_cacOpenset import train_osr
from eval_cacOpenset import eval_osr

In [5]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm.auto import tqdm

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# remove slow mirror from list of MNIST mirrors
# torchvision.datasets.MNIST.mirrors = [mirror for mirror in torchvision.datasets.MNIST.mirrors
#                                       if not mirror.startswith("http://yann.lecun.com")]

cuda:0


In [17]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

config = dict(
    epochs=2,
    alpha_magnitude = 10,
    num_classes=10,
    batch_size=128,
    learning_rate=0.001,
    weight_decay=5e-6,
    lbda_anchor_loss=0.1,
    train_ratio = 0.8,
    resume=False,
    init_weights=True,
    dataset="MNIST",
    architecture="CNN",
    use_wandb_flag=True,
    project_name="GMDL_PROJ",
    run_name='run1',
    trial=0)

## Loaders

In [7]:
def get_data(data_name='MNIST'):
  # Define the mandatory transformations
    mnist_mean = 0.1307
    mnist_std = 0.3081
    mnist_flip = 0
    mnist_rotate = 0
    mnist_scale_min = 0.7
    if data_name == 'MNIST':
        scale_min = mnist_scale_min
        flip = mnist_flip
        rotate = mnist_rotate
        means = mnist_mean
        stds = mnist_std
        transform_dict = {
                'train': transforms.Compose([
                    transforms.Resize((28,28)),
                    transforms.RandomResizedCrop((28,28), scale = (scale_min, 1.0)),
                    transforms.RandomHorizontalFlip(flip),
                    transforms.RandomRotation(rotate),
                    transforms.ToTensor(),
                    transforms.Normalize(means, stds)
                ]),
                'val': transforms.Compose([
                    transforms.Resize((28,28)),
                    transforms.ToTensor(),
                    transforms.Normalize(means, stds)
                ]),
                'test': transforms.Compose([
                    transforms.Resize((28,28)),
                    transforms.ToTensor(),
                    transforms.Normalize(means, stds)
                ])
            }
    else:
        transform_ood = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)) # OOD mean and std
    ])
    
    train_dataset, val_dataset, test_dataset =None, None, None
    if data_name == 'MNIST':
        train_dataset = torchvision.datasets.MNIST(root="./data",
                                                  train=True,
                                                  transform=transform_dict['train'],
                                                  download=True)
        val_dataset = torchvision.datasets.MNIST(root="./data",
                                                  train=True,
                                                  transform=transform_dict['val'],
                                                  download=True)
        test_dataset = torchvision.datasets.MNIST(root="./data",
                                                  train=False,
                                                  transform=transform_dict['test'],
                                                  download=True)
    elif data_name == 'FMNIST':
        test_dataset = torchvision.datasets.FashionMNIST(root="./data",
                                                train=False,
                                                transform=transform_ood,
                                                download=True)
    else:
        # CIFAR10
        test_dataset = torchvision.datasets.CIFAR10(root="./data",
                                                  train=False,
                                                  transform=transform_ood,
                                                  download=True)


    return train_dataset, val_dataset, test_dataset


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

def make_loaders(config, data_name='MNIST'):
    # CIFAR10 and FMNIST return only test_dataset
    # Make the data
    train_dataset, val_dataset, test_dataset = get_data(data_name)
    train_loader = make_loader(train_dataset, batch_size=config.batch_size)
    val_loader = make_loader(val_dataset, batch_size=config.batch_size)
    test_loader = make_loader(test_dataset, batch_size=config.batch_size)
    return train_loader, val_loader, test_loader

## OSR Model

In [55]:
def make_OSR_model(init_weights_flag):
    print('==> Building Network..')
    model = openSetClassifier(init_weights_flag)
    return model

## Model Pipeline

In [40]:
def model_pipeline_aux(hyperparameters, use_wandb_flag=False, wandb=None):
    if use_wandb_flag:
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config
    else:
      config = AttrDict(hyperparameters)

    # make the model, data, and optimization problem
    train_loader, val_loader, test_loader = make_loaders(config)
    model = make_OSR_model(config.init_weights)
    optimizer = optim.SGD(model.parameters(), lr = config.learning_rate, 
								momentum = 0.9, weight_decay = config.weight_decay)

    best_model_wts = train_osr(train_loader, val_loader, model, device, config, optimizer, wandb)


    # and test its final performance
    best_model = make_OSR_model(False)
    best_model.load_state_dict(best_model_wts) # load the best model for validation
    eval_osr(train_loader, test_loader, best_model, device, config, wandb)

    return best_model

In [41]:
def model_pipeline(hyperparameters):
  use_wandb_flag = hyperparameters['use_wandb_flag']
  if use_wandb_flag:
    with wandb.init(project=hyperparameters['project_name'], name=hyperparameters['run_name'], config=hyperparameters):
      return model_pipeline_aux(hyperparameters, use_wandb_flag, wandb)
  else:
      return model_pipeline_aux(hyperparameters, use_wandb_flag)

In [42]:
m = model_pipeline(config)

==> Building Anchores..

Epoch: 1

Epoch inside: 1
 [================================================================>]  Step: 43ms | Tot: 29s757ms | Loss: 1.308 | Acc: 80.670% (48402/6000 469/469 ...................................]  Step: 61ms | Tot: 4s151ms | Loss: 2.395 | Acc: 48.290% (4265/883 69/469 .....]  Step: 59ms | Tot: 5s49ms | Loss: 2.275 | Acc: 52.137% (5539/1062 83/469 91/469 .]  Step: 71ms | Tot: 6s578ms | Loss: 2.111 | Acc: 57.083% (7745/1356 106/469 ............................]  Step: 63ms | Tot: 6s777ms | Loss: 2.095 | Acc: 57.619% (8039/1395 109/469 ==============>.................................................]  Step: 65ms | Tot: 6s842ms | Loss: 2.089 | Acc: 57.777% (8135/1408 110/469 120/469 ..................................]  Step: 63ms | Tot: 8s159ms | Loss: 1.989 | Acc: 60.968% (10067/1651 129/469 .]  Step: 71ms | Tot: 8s359ms | Loss: 1.972 | Acc: 61.500% (10391/1689 132/469 .........................................]  Step: 66ms | Tot: 8s813ms | Loss: 1.941

epoch,▁█
train/CAC_Loss,█▆▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▂▁▂▁▂▂▁▂▁▁
train/anchor_Loss,█▇▅▅▅▄▄▄▄▄▃▄▃▄▃▃▃▂▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂
train/tuplet_Loss,█▆▄▄▃▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁▂▁▁▁▂▁▁▂▁▁
epoch,2
train/CAC_Loss,0.60055
train/anchor_Loss,5.06698
train/tuplet_Loss,0.09385
